In [1]:
#General dependencies (some of these may not be used - check at the end)
import pickle
import struct
import numpy as np
import scipy
import scipy.io as sio
from os.path import dirname, join as pjoin
import time
import sys
import matplotlib.pyplot as plt
%matplotlib tk
import random
from itertools import count
import pandas as pd
from matplotlib.animation import FuncAnimation
np.set_printoptions(threshold=np.inf)
plt.rcParams['animation.html']='jshtml'

import glob
import os

In [34]:
func(data_dir[5]) - func(data_dir[-1])

0

In [55]:
data_dir

['2021-08-10-0/data000/data000_0.mat',
 '2021-08-10-0/data000/data000_1.mat',
 '2021-08-10-0/data000/data000_10.mat',
 '2021-08-10-0/data000/data000_100.mat',
 '2021-08-10-0/data000/data000_101.mat',
 '2021-08-10-0/data000/data000_102.mat',
 '2021-08-10-0/data000/data000_103.mat',
 '2021-08-10-0/data000/data000_104.mat',
 '2021-08-10-0/data000/data000_105.mat',
 '2021-08-10-0/data000/data000_106.mat',
 '2021-08-10-0/data000/data000_107.mat',
 '2021-08-10-0/data000/data000_108.mat',
 '2021-08-10-0/data000/data000_109.mat',
 '2021-08-10-0/data000/data000_11.mat',
 '2021-08-10-0/data000/data000_110.mat',
 '2021-08-10-0/data000/data000_111.mat',
 '2021-08-10-0/data000/data000_112.mat',
 '2021-08-10-0/data000/data000_113.mat',
 '2021-08-10-0/data000/data000_114.mat',
 '2021-08-10-0/data000/data000_115.mat',
 '2021-08-10-0/data000/data000_116.mat',
 '2021-08-10-0/data000/data000_117.mat',
 '2021-08-10-0/data000/data000_118.mat',
 '2021-08-10-0/data000/data000_119.mat',
 '2021-08-10-0/data000

In [54]:
print(os.stat(data_dir[0]).st_ctime_ns)
print(os.stat(data_dir[1]).st_ctime_ns)
print(os.stat(data_dir[2]).st_ctime_ns)

1628636130000000000
1628636130000000000
1628636131000000000


In [7]:
# Load Data in Real Time Continuously

# Initialize the Path where data is stored
path = r'/Shakedown/Users/vision/Desktop/Xilinx FPGA TCP Server Code'

date_piece = '2021-08-10-0'
file_num = 0
run = 0

datarun = 'data'+str(run).zfill(3)
folder = datarun+'_'+str(file_num)
file_type = '/*mat'
latest_file_pre = ' '

def func(p):
    return int(((p.split('.')[0]).split('_'))[-1])


while not(os.path.isdir(pjoin(date_piece, datarun))):
    continue
    
    
while True:
    data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)
    if len(data_dir)>0:
        break

# Continuously Check for New Data
while True:
    
    data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)
    
    # Look at the Newest Data File in the Directory
    #time.sleep(0.1)
    latest_file = max(data_dir, key=func)
    
    # If it is the same data as last time, leave it and keep checking
    if latest_file == latest_file_pre:
        continue
    
    print(latest_file)
    # If is is new data, update the check for next time
    latest_file_pre = latest_file
    
    # In the off chance the file has been written, but not saved by 
    # the TCP socket yet, pause
    time.sleep(0.1)
    
    # Load the newest data
    mat_contents = sio.loadmat(latest_file)
    data = mat_contents['dummy_data_ch586_hard_gnd'][0][:]
    
    # Convert the data
    data_real = np.zeros((32,32,len(data)-2))  #Data should be a power/divisible by 2^N where N= number of channels active in the recording buffer. Note Divide by 2 because of the double count and -1 because the first sample (w/o double count) is garbo
    cnt_real = np.zeros((32,32,len(data)-2))
    #counter = np.zeros((1,1,len(data)-2)) - Uncomment once data w/ counter
    
    # Initialize Variables for the Loop
    cnt_pre = 0
    N = 0 # Sample times
    chan_index_pre = 1025 #Make sure it is not a true channel
    #counter_pre = 0 - Uncomment once data w/ counter
    
    for i in range(2,len(data)-1): 
        # Convert bit number into binary
        word = (np.binary_repr(data[i],32))
        
        # Break that binary into it's respective pieces and convert to bit number
        cnt = int(word[12:14],2)
        col = int(word[27:32],2)
        row = int(word[22:27],2)
        chan_index = row*32 + col
        #count_all = int(word[1:12],2) - Uncomment once data w/ counter
        
        # Only record the unique non-double count sample
        if(i==2 or (cnt_pre != cnt or chan_index != chan_index_pre)):
            
            # Sample time only changes when cnt changes
            if cnt != cnt_pre:
                N += 1
                
            # On the occurance the first cnt is not 0, make sure sample time is 0
            if i == 2:
                N = 0
                    
            # Uncomment this once data with counter comes in
            #if counter_pre != count_all:
                #count[0,0,counter_pre] = count_all
                
            # Update variables
            cnt_pre = cnt
            chan_index_pre = chan_index
            
            # Record pertinent data
            data_real[row][col][N] = int(word[14:22],2)
            cnt_real[row][col][N] = cnt
            
    # Truncate data to be the number of samples       
    num_samples = N
    data_real = data_real[:,:,:N]
    cnt_real = cnt_real[:,:,:N]
    #total_time = count_all*0.05 - Uncomment once data w/ counter
    
    plt.close('all')

    # Initialize variables
    chan_idx = int(587) #The electrode of interest
    max_cnt = 20 #Saturation point for number of samples
    #max_amp = 225 #Saturation point for avg amplitude of samples - Not in use
    max_dot = 600 #Saturation point for dot size
    time_win = 30 #Number of samples of interest (i.e. 3ms/0.05 ms per sample = 60 samples)

    # Initialize Electrode Indices
    chan_row = int(chan_idx / 32)
    chan_col = chan_idx - (chan_row * 32) - 1

    # Determine the Time of Each Sample (NOTE: Once counter is in place, this needs adjusted so any non-collision free samples appear void of values)
    total_time = num_samples*0.05 #Sampling rate 1/0.05 ms 
    times = np.linspace(0,total_time,num_samples)

    # Initialize figure with subplots
    fig, ax = plt.subplots(2,1,figsize=(9,10),gridspec_kw={'height_ratios':[2,1]})
    fig.show()
    plt.style.use('fivethirtyeight')
    plt.grid(True)
    plt.subplots_adjust(left=0.4)
    ax[0].set_title('Whole Array Activity')
    ax[1].set_title('Electrode # ' + str(chan_idx))
    ax[1].set(xlabel = 'Time (ms)', ylabel = 'Voltage')
    textstr2 = 'Time Window = %.2f ms' %(time_win*0.05)

    # Initialize Array Indices
    base = np.arange(1,33)
    rows = [ele for ele in base for i in range(32)]
    temp = [1 for i in range(32)]
    cols = [ele * tele for tele in temp for ele in base]

    # Initialize Variables for the Loop (size and colors will be returned) 
    size = np.zeros((32,32,0)) # For each dot
    num_sam = np.zeros((32,32,1))
    colors = np.zeros((32,32,0)) # For each dot
    avg_val = np.zeros((32,32,1))
    max_val_temp = np.zeros((2,1))
    max_val = np.zeros((2,0))
    max_num_temp = np.zeros((2,1))
    max_num = np.zeros((2,0))
    num_chan = np.zeros((0))
    num_chan_temp = np.zeros((1))
    i = 0
    print('starting loop')

    # Calculate plotting values over entire given sample space
#     while i < num_samples:

#         # Determine the non-zero collection of samples in time window and max number of samples in time window
#         num_sam[:,:,0] = np.count_nonzero(data_real[:,:,i:i+time_win],axis=2)
#         num_chan_temp[0] = np.count_nonzero(num_sam[:,:,0])
#         num_chan = np.append(num_chan,num_chan_temp,axis=0)
#         max_num_temp[0,0] = np.max(num_sam[:,:,0])
#         max_num_temp[1,0] = np.argmax(num_sam[:,:,0])
#         max_num = np.append(max_num,max_num_temp,axis=1)

#         # Determine average value of samples in time window and max value of samples in time window
#         # NOTE: May alter average to be the average of non-zero samples
#         avg_val[:,:,0] = np.mean(data_real[:,:,i:i+time_win],axis=2)
#         max_val_temp[0,0] = np.max(avg_val[:,:,0])
#         max_val_temp[1,0] = np.argmax(avg_val[:,:,0])
#         max_val = np.append(max_val,max_val_temp,axis=1)

#         # Record the average values to represent colors for each time window
#         # NOTE: Using append b/c it does not require exact calculations. May be a faster version of append (i.e. concatenate)
#         colors = np.append(colors,avg_val,axis=2)

#         # Scale size according to the max value unless that value exceeds the saturation point. Squaring in the scale to emphasize differences
#         if max_num_temp[0,0] < max_cnt:
#             scale = (max_dot - 10) / (max_num_temp[0,0]**2)
#             size = np.append(size, num_sam*scale + 10, axis = 2)
#         elif max_num_temp[0,0] >= max_cnt:
#             scale = (max_dot - 10) / (max_cnt**2)
#             size = np.append(size, num_sam*scale + 10, axis = 2)

#         # Update i by the time_window
#         i += time_win
        
    print('done with loop')
        
        # STEP 2: ACTIVATE FIGURE
    # Note: This can be applied at the end of the last cell, but Jupyter
    # Notebooks gets angry about this even though the plotting goes on as normal

    # NOTE: This needs updated to plot the most recent of real-time data
    # and to hold on the last figure of true data while we are waiting for 
    # another file

    # Initialize every time
#     x1 = rows
#     y1 = cols
    x2 = np.empty(0)
    y2 = np.empty(0)
    i = 0
    n = 0

    # Plot for the entire length of incoming data
    while num_samples > i:

        # For the electrode plot, append the new data to be plotted
        x2 = np.append(x2,times[i:i+time_win])
        y2 = np.append(y2, data_real[chan_row, chan_col, i:i+time_win])

        # For the scatter plot, update the size and colors
#         ax[0].scatter(x1,y1,size[:,:,n],c=colors[:,:,n],cmap='jet')
#         ax[0].set_title('Whole Array Activity')

        # For the electrode plot, set a new xlimit
        ax[1].plot(x2,y2,color='r')
        ax[1].set_xlim(left=max(0,int(x2[i])-5),right=(int(x2[i])+time_win*0.05+1))

        # Text Box of the Electrode with Maximum Amplitude
#         textstr = 'Max Avg Amp\nElec = %04d\nAmp = %03d\n\nMax Num Samp\nElec = %04d\nNum=%02d\n\n\nNum Channels: \n%03d'%(max_val[1,n],max_val[0,n],max_num[1,n],max_num[0,n],num_chan[n])
#         ax[0].text(-0.4,0.25,textstr,transform=ax[0].transAxes,fontsize=14)
#         ax[0].text(-0.4, 1, textstr2,transform=ax[0].transAxes,fontsize=14)



        # This step is fast but most time consuming - may replace with fig.canvas.update()
        fig.canvas.draw()

        # Flush events (equivalent to time.sleep or plt.pause but faster)
        fig.canvas.flush_events()

        # Use time.sleep if you want to plot really slowly you weirdo
        #time.sleep(1)

        # Clear the scatter plot
#         ax[0].clear()

        # Update the variables
        i += time_win
        n += 1


    # STEP 3: UPDATE FILE NUMBER
    file_num += 1
    
        
        
# NOTE: Put in interrupt on updating plot to look for new data coming in
# Only plot the last 1sec of data
# Scroll bar 
# Speed scatter plot loop






2021-08-10-0/data000/data000_0.mat
starting loop
done with loop


TclError: invalid command name "pyimage60"

In [57]:
print(latest_file)

2021-08-10-0/data000/data000_5.mat


In [ ]:
# # Load Data & Convert to Real Values

# # Insert Real Time loading here

# #load matfile
# #extract matcontents as variable data
# #data000+str(cnt)

# #Initialize File Number
# path = '/Shakedown/Users/vision' # Change this as necessary
# date_piece = '2021-08-10-0'
# file_num = 0

# #Get filename from data directory
# data_dir = pjoin(dirname(path),date_piece,'data000'+str(file_num),'data000'+str(file_num)+'.mat')

# # Load Data from Data Directory
# mat_contents = sio.loadmat(data_dir)
# data = mat_contents['vect'][0][:]


# #NOTE: Eventually I want this to be constantly checking if a new file
# # has been written to the directory and plot the latest file

# # Data Conversion

# # Removing double count from the data and interpreting data
# data_real = np.zeros((32,32,len(data)-2))  #Data should be a power/divisible by 2^N where N= number of channels active in the recording buffer. Note Divide by 2 because of the double count and -1 because the first sample (w/o double count) is garbo
# cnt_real = np.zeros((32,32,len(data)-2))
# #counter = np.zeros((1,1,len(data)-2)) - Uncomment once data w/ counter

# # Initialize variables for the loop
# cnt_pre = 0
# N = 0 # Sample times
# chan_index_pre = 1025 #Make sure it is not a true channel
# #counter_pre = 0 - Uncomment once data w/ counter

# for i in range(2,len(data)-1): 
#     # Convert bit number into binary
#     word = (np.binary_repr(data[i],32))
    
#     # Break that binary into it's respective pieces and convert to bit number
#     cnt = int(word[12:14],2)
#     col = int(word[27:32],2)
#     row = int(word[22:27],2)
#     chan_index = row*32 + col
#     #count_all = int(word[1:12],2) - Uncomment once data w/ counter
    
#     # Only record the unique non-double count sample
#     if(i==2 or (cnt_pre != cnt or chan_index != chan_index_pre)):
        
#         # Sample time only changes when cnt changes
#         if cnt != cnt_pre:
#             N += 1
        
#         # On the occurance the first cnt is not 0, make sure sample time is 0
#         if i == 2:
#             N = 0
        
#         # Uncomment this once data with counter comes in
#         #if counter_pre != count_all:
#             #count[0,0,counter_pre] = count_all
        
#         # Update variables
#         cnt_pre = cnt
#         chan_index_pre = chan_index
        
#         # Record pertinent data
#         data_real[row][col][N] = int(word[14:22],2)
#         cnt_real[row][col][N] = cnt
        
# # Truncate data to be the number of samples        
# num_samples = N
# data_real = data_real[:,:,:N]
# cnt_real = cnt_real[:,:,:N]
# #total_time = count_all*0.05 - Uncomment once data w/ counter

In [ ]:
# Plot Data
# GOAL: Array Plot w/ Electrode Plots
# STEP 1: INTIALIZE FIGURE

plt.close('all')

# Initialize variables
chan_idx = int(283) #The electrode of interest
max_cnt = 20 #Saturation point for number of samples
#max_amp = 225 #Saturation point for avg amplitude of samples - Not in use
max_dot = 600 #Saturation point for dot size
time_win = 10 #Number of samples of interest (i.e. 3ms/0.05 ms per sample = 60 samples)

# Initialize Electrode Indices
chan_row = int(chan_idx / 32)
chan_col = chan_idx - (chan_row * 32) - 1

# Determine the Time of Each Sample (NOTE: Once counter is in place, this needs adjusted so any non-collision free samples appear void of values)
total_time = num_samples*0.05 #Sampling rate 1/0.05 ms 
times = np.linspace(0,total_time,num_samples)

# Initialize figure with subplots
fig, ax = plt.subplots(2,1,figsize=(9,10),gridspec_kw={'height_ratios':[2,1]})
fig.show()
plt.style.use('fivethirtyeight')
plt.grid(True)
plt.subplots_adjust(left=0.4)
ax[0].set_title('Whole Array Activity')
ax[1].set_title('Electrode # ' + str(chan_idx))
ax[1].set(xlabel = 'Time (ms)', ylabel = 'Voltage')
textstr2 = 'Time Window = %.2f ms' %(time_win*0.05)

# Initialize Array Indices
base = np.arange(1,33)
rows = [ele for ele in base for i in range(32)]
temp = [1 for i in range(32)]
cols = [ele * tele for tele in temp for ele in base]

# Initialize Variables for the Loop (size and colors will be returned) 
size = np.zeros((32,32,0)) # For each dot
num_sam = np.zeros((32,32,1))
colors = np.zeros((32,32,0)) # For each dot
avg_val = np.zeros((32,32,1))
max_val_temp = np.zeros((2,1))
max_val = np.zeros((2,0))
max_num_temp = np.zeros((2,1))
max_num = np.zeros((2,0))
num_chan = np.zeros((0))
num_chan_temp = np.zeros((1))
i = 0

# Calculate plotting values over entire given sample space
while i < num_samples:
    
    # Determine the non-zero collection of samples in time window and max number of samples in time window
    num_sam[:,:,0] = np.count_nonzero(data_real[:,:,i:i+time_win],axis=2)
    num_chan_temp[0] = np.count_nonzero(num_sam[:,:,0])
    num_chan = np.append(num_chan,num_chan_temp,axis=0)
    max_num_temp[0,0] = np.max(num_sam[:,:,0])
    max_num_temp[1,0] = np.argmax(num_sam[:,:,0])
    max_num = np.append(max_num,max_num_temp,axis=1)
    
    # Determine average value of samples in time window and max value of samples in time window
    # NOTE: May alter average to be the average of non-zero samples
    avg_val[:,:,0] = np.mean(data_real[:,:,i:i+time_win],axis=2)
    max_val_temp[0,0] = np.max(avg_val[:,:,0])
    max_val_temp[1,0] = np.argmax(avg_val[:,:,0])
    max_val = np.append(max_val,max_val_temp,axis=1)
    
    # Record the average values to represent colors for each time window
    # NOTE: Using append b/c it does not require exact calculations. May be a faster version of append (i.e. concatenate)
    colors = np.append(colors,avg_val,axis=2)
    
    # Scale size according to the max value unless that value exceeds the saturation point. Squaring in the scale to emphasize differences
    if max_num_temp[0,0] < max_cnt:
        scale = (max_dot - 10) / (max_num_temp[0,0]**2)
        size = np.append(size, num_sam*scale + 10, axis = 2)
    elif max_num_temp[0,0] >= max_cnt:
        scale = (max_dot - 10) / (max_cnt**2)
        size = np.append(size, num_sam*scale + 10, axis = 2)
    
    # Update i by the time_window
    i += time_win




In [ ]:
# STEP 2: ACTIVATE FIGURE
# Note: This can be applied at the end of the last cell, but Jupyter
# Notebooks gets angry about this even though the plotting goes on as normal

# NOTE: This needs updated to plot the most recent of real-time data
# and to hold on the last figure of true data while we are waiting for 
# another file

# Initialize every time
x1 = rows
y1 = cols
x2 = np.empty(0)
y2 = np.empty(0)
i = 0
n = 0

# Plot for the entire length of incoming data
while num_samples > i:
    
    # For the electrode plot, append the new data to be plotted
    x2 = np.append(x2,times[i:i+time_win])
    y2 = np.append(y2, data_real[chan_row, chan_col, i:i+time_win])
    
    # For the scatter plot, update the size and colors
    ax[0].scatter(x1,y1,size[:,:,n],c=colors[:,:,n],cmap='jet')
    ax[0].set_title('Whole Array Activity')
    
    # For the electrode plot, set a new xlimit
    ax[1].plot(x2,y2,color='r')
    ax[1].set_xlim(left=max(0,int(x2[i])-5),right=(int(x2[i])+time_win*0.05+1))
    
    # Text Box of the Electrode with Maximum Amplitude
    textstr = 'Max Avg Amp\nElec = %04d\nAmp = %03d\n\nMax Num Samp\nElec = %04d\nNum=%02d\n\n\nNum Channels: \n%03d'%(max_val[1,n],max_val[0,n],max_num[1,n],max_num[0,n],num_chan[n])
    ax[0].text(-0.4,0.25,textstr,transform=ax[0].transAxes,fontsize=14)
    ax[0].text(-0.4, 1, textstr2,transform=ax[0].transAxes,fontsize=14)


    
    # This step is fast but most time consuming - may replace with fig.canvas.update()
    fig.canvas.draw()
    
    # Flush events (equivalent to time.sleep or plt.pause but faster)
    fig.canvas.flush_events()
    
    # Use time.sleep if you want to plot really slowly you weirdo
    #time.sleep(1)
    
    # Clear the scatter plot
    ax[0].clear()
    
    # Update the variables
    i += time_win
    n += 1
    
    
# STEP 3: UPDATE FILE NUMBER
file_num += 1
    